# CodeGen

Author: Srijan, Ayush, Pavan


## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/mnt/', force_remount=True)

Mounted at /content/mnt/


## Data Processing

In [ ]:
DIR='/content/mnt/My Drive/codeGen'

In [ ]:
from os import listdir
import tarfile
for files in listdir(DIR): print(files)

# extract data to session
with tarfile.open(f'{DIR}/htmlData.tar.gz', 'r:gz') as tref:
    tref.extractall()

htmlData.tar.gz


In [ ]:
!pip install -q spacy 

In [ ]:
import spacy
from spacy.tokens import Token
from spacy.symbols import ORTH
Token.set_extension('tag', default=False)

In [ ]:
def create_custom_tokenizer(nlp):
    from spacy import util
    from spacy.tokenizer import Tokenizer
    from spacy.lang.tokenizer_exceptions import TOKEN_MATCH
    prefixes =  nlp.Defaults.prefixes + ('^<i>',)
    suffixes =  nlp.Defaults.suffixes + ('</i>$',)
    # remove the tag symbols from prefixes and suffixes
    prefixes = list(prefixes)
    prefixes.remove('<')
    prefixes = tuple(prefixes)
    suffixes = list(suffixes)
    suffixes.remove('>')
    suffixes = tuple(suffixes)
    infixes = nlp.Defaults.infixes
    rules = nlp.Defaults.tokenizer_exceptions
    token_match = TOKEN_MATCH
    prefix_search = (util.compile_prefix_regex(prefixes).search)
    suffix_search = (util.compile_suffix_regex(suffixes).search)
    infix_finditer = (util.compile_infix_regex(infixes).finditer)
    return Tokenizer(nlp.vocab, rules=rules,
                     prefix_search=prefix_search,
                     suffix_search=suffix_search,
                     infix_finditer=infix_finditer,
                     token_match=token_match)

In [ ]:
html_tags = ['a', 'abbr', 'address', 'area', 'article',  'aside', 'audio', 'b', 'base', 'bdi', 'bdo', 'blockquote', 'body', 'br', 'button', 'canvas', 'caption', 
             'cite', 'code', 'col', 'colgroup', 'data', 'datalist', 'dd', 'del', 'details', 'dfn', 'dialog', 'div', 'dl', 'dt', 'em', 'embed', 'fieldset', 'figure', 'footer',
             'form', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'head', 'header', 'hgroup', 'hr', 'html', 'i', 'iframe', 'img', 'input', 'ins', 'kbd', 'keygen', 'label', 'legend',
             'li', 'link', 'main', 'map', 'mark', 'menu', 'menuitem', 'meta', 'meter', 'nav', 'noscript', 'object', 'ol', 'optgroup', 'option', 'output', 'p', 'param', 
             'pre', 'progress', 'q', 'rb', 'rp', 'rt', 'rtc', 'ruby', 's', 'samp', 'script', 'section', 'select', 'small', 'source', 'span', 'strong', 'style', 'sub', 'summary',
             'sup', 'table', 'tbody', 'td', 'template', 'textarea', 'tfoot', 'th', 'thead', 'time', 'title', 'tr', 'track','u','ul','var','video','wbr' ]

In [ ]:
nlp = spacy.load('en_core_web_sm')
tokenizer = create_custom_tokenizer(nlp)
nlp.tokenizer = tokenizer

for tagName in html_tags:
    nlp.tokenizer.add_special_case(f"<{tagName}>", [{ORTH: f"<{tagName}>"}])    
    nlp.tokenizer.add_special_case(f"</{tagName}>", [{ORTH: f"</{tagName}>"}])    

for tagName in "br p hr".split():
    nlp.tokenizer.add_special_case(f"<{tagName}/>", [{ORTH: f"<{tagName}/>"}])    

In [ ]:
import codecs
import re

def sanitize_text(text):
    ans = ''
    text = text.replace('\xa0', ' ')
    for line in text.split('\n'):
        line = re.sub('^\/\*.*\*\/', ' ', line)
        line = line.strip()
        match = re.match("\<.+\>.*\<\/.+\>", line)
        if match:
            line = re.sub('\>', '> ', line)
            line = re.sub('\<', ' <', line)
        ans += line + ' '
    return ans

files_token = []
for file in listdir('code'):
    with codecs.open(f'code/{file}', 'r', 'utf-8') as fp: 
        text = sanitize_text(fp.read())
        doc = nlp(text)
        # print(text)
        files_token.append([e.text for e in doc if e.text.strip() != ''])

In [ ]:
print(files_token[1])

['<!DOCTYPE', 'html>', '<html>', '<body>', '<p>', 'This', 'is', 'a', 'paragraph', '.', '</p>', '<p>', 'This', 'is', 'another', 'paragraph', '.', '</p>', '</body>', '</html>']


In [ ]:
from sklearn.model_selection import train_test_split

print(f"total data: {len(files_token)} items")


total data: 834 items


In [ ]:
train_iter,test_iter = train_test_split(files_token,test_size=.20,random_state=60)
#testing
print("Train - ",train_iter[1])
print("Test - ",test_iter[1])

Train -  ['<!DOCTYPE', 'html>', '<html', 'lang="en">', '<head>', '<title>', 'Bootstrap', 'Example', '</title>', '<meta', 'charset="utf-8">', '<meta', 'name="viewport', '"', 'content="width', '=', 'device', '-', 'width', ',', 'initial', '-', 'scale=1">', '<link', 'rel="stylesheet', '"', 'href="https://maxcdn.bootstrapcdn.com', '/', 'bootstrap/3.4.1', '/', 'css', '/', 'bootstrap.min.css">', '<script', 'src="https://ajax.googleapis.com', '/', 'ajax', '/', 'libs', '/', 'jquery/3.5.1', '/', 'jquery.min.js">', '</script>', '<script', 'src="https://maxcdn.bootstrapcdn.com', '/', 'bootstrap/3.4.1', '/', 'js', '/', 'bootstrap.min.js">', '</script>', '</head>', '<body>', '<div', 'class="container">', '<h2>', 'Labels', '</h2>', '<h1>', 'Example', '<span', 'class="label', 'label', '-', 'default">', 'New', '</span>', '</h1>', '<h2>', 'Example', '<span', 'class="label', 'label', '-', 'default">', 'New', '</span>', '</h2>', '<h3>', 'Example', '<span', 'class="label', 'label', '-', 'default">', 'New',

## Transformer Implementation

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
import io
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

# train_iter = WikiText2(split='train')
# tokenizer = get_tokenizer('basic_english')
print(train_iter)
counter = Counter()
for line in train_iter:
    counter.update(line)
print(counter)
print("Dictionary len",len(counter))
vocab = Vocab(counter)
for item in train_iter:
  for token in item:
    print(vocab[token],end=' ')
print("\n")
for item in test_iter:
  for token in item:
    print(vocab[token],end=' ')
print("\n")

def data_process(raw_text_iter):
  data = [torch.tensor([vocab[token] for token in item],
                       dtype=torch.long) for item in raw_text_iter]
  return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
print(train_data)
# val_data = data_process(val_iter)
test_data = data_process(test_iter)
print(test_data)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 10
eval_batch_size = 2
train_data = batchify(train_data, batch_size)
# val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)
print(train_data)
print(test_data)

[['<!DOCTYPE', 'html>', '<html>', '<head>', '<style>', 'a', ':', 'link', '{', 'background', '-', 'color', ':', 'yellow', ';', '}', 'a', ':', 'visited', '{', 'background', '-', 'color', ':', 'cyan', ';', '}', 'a', ':', 'hover', '{', 'background', '-', 'color', ':', 'lightgreen', ';', '}', 'a', ':', 'active', '{', 'background', '-', 'color', ':', 'hotpink', ';', '}', '</style>', '</head>', '<body>', '<h2>', 'CSS', 'Links', '</h2>', '<p>', '<b>', '<a', 'href="default.asp', '"', 'target="_blank">', 'This', 'is', 'a', 'link', '</a>', '</b>', '</p>', '<p>', '<b>', 'Note', ':', '</b>', 'a', ':', 'hover', 'MUST', 'come', 'after', 'a', ':', 'link', 'and', 'a', ':', 'visited', 'in', 'the', 'CSS', 'definition', 'in', 'order', 'to', 'be', 'effective', '.', '</p>', '<p>', '<b>', 'Note', ':', '</b>', 'a', ':', 'active', 'MUST', 'come', 'after', 'a', ':', 'hover', 'in', 'the', 'CSS', 'definition', 'in', 'order', 'to', 'be', 'effective', '.', '</p>', '</body>', '</html>'], ['<!DOCTYPE', 'html>', '<htm

In [ ]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [ ]:
ntokens = len(vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
softmax = nn.Softmax(dim=1)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [ ]:
best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    test_loss = evaluate(model, test_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | test loss {:5.2f} | '
          'test ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     test_loss, math.exp(test_loss)))
    print('-' * 89)

    if test_loss < best_val_loss:
        best_val_loss = test_loss
        best_model = model

    scheduler.step()

-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  1.37s | test loss  3.20 | test ppl    24.53
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time:  1.15s | test loss  2.88 | test ppl    17.85
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time:  1.14s | test loss  2.70 | test ppl    14.89
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   4 | time:  1.15s | test loss  2.71 | test ppl    15.07
----------------------------------------------------------------------------------

In [ ]:
print(best_model)

In [ ]:
test_loss = evaluate(best_model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  3.15 | test ppl    23.38


In [ ]:
print(evaluate(best_model,test_data))

3.1519810035824776


In [ ]:
# def evaluate_ans(sentence, max_length=40):
  # inp sentence is portuguese, hence adding the start and end token
html="<html>"

html_san=sanitize_text(html)
doc_san=nlp(sanitize_text(html))
tokens=[e.text for e in doc_san if e.text.strip() != '']

counter = Counter(tokens)
print(counter)
vocab1=Vocab(counter)
print(vocab1)
tok_tensor=torch.tensor([vocab[token] for token in tokens],dtype=torch.long)
print(tok_tensor)

Counter({'<html>': 1})
tensor([17])


In [ ]:
tok_tensor_data = batchify(tok_tensor, 2)
print(tok_tensor_data)


def evaluate1(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
            # print(f"{output_flat} \t\t {targets}\n\n")
            # print(f"{softmax(output_flat)} \t\t {targets[0]}\n\n")
            predicted = torch.argmax(softmax(output_flat), dim=1)
            pred = [vocab.itos[w] for w in predicted]
            ans = [vocab.itos[w] for w in targets]
            print(f"predicted:\n{pred} \n\n target:\n{ans} \n\n\n")
            print("Predicted:\n")
            # for item in pred:
            #   print(item)
            # print("\n")
            # print("Target:\n")
            # for item in ans:
            #   print(item)

    return total_loss / (len(data_source) - 1)


print(evaluate1(best_model,tok_tensor_data))

tensor([], device='cuda:0', size=(0, 2), dtype=torch.int64)
-0.0


In [ ]:
test_loss = evaluate1(best_model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

predicted:
['html>', 'is', '<html>', 'a', '<head>', '1', '<h1>', '</h2>', 'The', '<p>', 'input', 'This', 'min', 'is', '</h1>', 'an', '</h1>', '2', '<h2>', '</h2>', 'The', '<p>', 'link', 'This', 'and', 'is', 'specifies', 'a', 'the', 'paragraph', 'link', '.', 'text', '</p>', 'and', '<p>', 'the', 'This', 'first', 'the', 'field', 'font', '(', 'size', 'name', 'size', 'of', 'in', 'the', 'em', '>', ')', 'is', 'it', '</div>', ')', '</body>', ')', 'action="/action_page.php">', 'paragraph', '<input', 'img', 'for="fname">', 'all', '"', 'the', '</th>', 'collapsible', '.', 'settlement'] 

 target:
['html>', 'is', '<html>', 'heading', '<body>', '1', '<h1>', '</h1>', 'The', '<h2>', 'input', 'This', '<unk>', 'is', 'attribute', 'heading', '</h1>', '2', '<p>', '</h2>', 'The', '<p>', '<unk>', 'This', 'attribute', 'is', 'specifies', 'a', 'a', 'paragraph', 'regular', '.', '<unk>', '</p>', 'that', '<p>', 'the', 'Specifying', 'input', 'the', 'element', 'font', "'s", '-', 'value', 'size', 'is', 'in', 'checked